In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from scipy.sparse import coo_matrix, hstack


In [2]:
#ugly but functional function
def create_file(name,mis_labels,agr_labels):
    indexes=[x for x in range(5001,6001)]
    data={'id':indexes,
    'misogynous':mis_labels,
    'aggressiveness':agr_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous','aggressiveness'])
    dataframe.to_csv(name,sep='\t',index=False)

# Run 1 constained tf-idf

In [3]:
DATAFRAME_train = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
DATAFRAME=pd.concat([DATAFRAME_train,DATAFRAME_test])

vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
    strip_accents='unicode', analyzer='word', token_pattern=r'\b[^\d\W]+\b',
    ngram_range=(1, 5),use_idf=True)


In [10]:
X=vectorizer.fit_transform(DATAFRAME['text'])
X_train,X_test=X[:5000],X[5000:]
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'])
result_agr=model.predict(X_test)



In [11]:
for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 #Sunt 4 cazuri cand se intampla

In [12]:
create_file('MDD.A.r.c.run1',result_mis,result_agr)

# Unconstrained Run 2

In [13]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)

embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [14]:
DATAFRAME_train = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
DATAFRAME=pd.concat([DATAFRAME_train,DATAFRAME_test])

In [15]:
X=DATAFRAME['text'].values
X_train,X_test=get_mean_embeddings(X[:5000],embeddings_index),get_mean_embeddings(X[5000:],embeddings_index)

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'])
result_agr=model.predict(X_test)

for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 #Sunt 4 cazuri cand se intampla

create_file('MDD.A.r.u.run2',result_mis,result_agr)

## Unconstrained run 3

In [3]:
DATAFRAME_train = pd.read_csv('different processed dataframes/normal_processed.tsv', sep='\t')
import it_core_news_lg
nlp=it_core_news_lg.load()

embeddingurile_train=[]
for text in DATAFRAME_train['text'].values:
    embeddingurile_train.append(nlp(text).vector)


embeddingurile_test=[]
DATAFRAME_test=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')
for text in DATAFRAME_test['text'].values:
    embeddingurile_test.append(nlp(text).vector)


In [4]:
X=DATAFRAME_train['clean'].values
X_train,X_test=np.array(embeddingurile_train),np.array(embeddingurile_test)
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)


model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test)
model.fit(X_train,DATAFRAME_train['aggressiveness'])
result_agr=model.predict(X_test)


for i,(mis,agr) in enumerate(zip(result_mis,result_agr)):
    if mis==0 and agr==1:
        result_agr[i]=0 
        
create_file('MDD.A.r.u.run3',result_mis,result_agr)

# TASK B RUN 2

In [6]:
DATAFRAME_train_synt = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_synt.tsv', sep='\t')
DATAFRAME_train_raw=pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
DATAFRAME_train=pd.concat([DATAFRAME_train_raw,DATAFRAME_train_synt],ignore_index=True)

DATAFRAME_test_synt=pd.read_csv('test/AMI2020_test_synt.tsv',sep='\t')
DATAFRAME_test_raw=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')


In [7]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)

embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [10]:

X_train,X_test_raw,X_test_synt=get_mean_embeddings(DATAFRAME_train['text'],embeddings_index),get_mean_embeddings(DATAFRAME_test_raw['text'],embeddings_index),get_mean_embeddings(DATAFRAME_test_synt['text'],embeddings_index)

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)

#Raw test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_raw)
create_file_2('MDD.B.r.u.run2',result_mis,'raw')

#Synt test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_synt)
create_file_2('MDD.B.s.u.run2',result_mis,'synt')


In [9]:
def create_file_2(name,mis_labels,data_type):
    if data_type=='raw':
        indexes=[x for x in range(5001,6001)]
    else:
        indexes=[x for x in range (2014,3922)]
    data={'id':indexes,
    'misogynous':mis_labels}
    dataframe=pd.DataFrame(data,index=indexes,columns=['id','misogynous'])
    dataframe.to_csv(name,sep='\t',index=False)

## TASK B RUN 3

In [21]:
DATAFRAME_train_synt = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_synt.tsv', sep='\t')
DATAFRAME_train_synt['clean']=DATAFRAME_train_synt['text']

DATAFRAME_train_raw=pd.read_csv('different processed dataframes/noun_chuncks,processed.tsv', sep='\t')

DATAFRAME_train=pd.concat([DATAFRAME_train_raw,DATAFRAME_train_synt],ignore_index=True)
DATAFRAME_train['clean']=DATAFRAME_train['clean'].apply(lambda x:str(x)) #le am pus ghena in tsv

DATAFRAME_test_synt=pd.read_csv('test/AMI2020_test_synt.tsv',sep='\t')
DATAFRAME_test_raw=pd.read_csv('test/AMI2020_test_raw.tsv',sep='\t')

In [17]:
DATAFRAME_train['clean']

0                                    fatti te lo ti bocca
1           tu semafori la lavavetri il vitalizio ti gola
2       amore skype ti i miei parenti un tablet i pare...
3       salvo la culona mosca aummaumm putin commesse ...
4       gentiloni danno la francia ci sarkosy la culon...
                              ...                        
7009                  sposa non dovrebbe essere soffocata
7010             ragazze non dovrebbero essere massacrate
7011                      mogli dovrebbero essere stimate
7012                                     moglie terribile
7013             signorina non dovrebbe essere massacrata
Name: clean, Length: 7014, dtype: object

In [24]:

X_train=get_mean_embeddings(DATAFRAME_train['clean'].values,embeddings_index)
X_test_raw,X_test_synt=get_mean_embeddings(DATAFRAME_test_raw['text'],embeddings_index),get_mean_embeddings(DATAFRAME_test_synt['text'],embeddings_index)
model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)


#Raw test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_raw)
create_file_2('MDD.B.r.u.run3',result_mis,'raw')

#Synt test
model.fit(X_train,DATAFRAME_train['misogynous'])
result_mis=model.predict(X_test_synt)
create_file_2('MDD.B.s.u.run3',result_mis,'synt')